# 共线性数据分析
* 回归系数极为敏感
* 预测精度和模型可靠性极大降低

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf
import scipy.stats as st

# 1.共线性

In [2]:
data = pd.read_csv("/Users/luofeifei/study/RegressionAnalysis/All_Data/P236.txt",sep="\t")

In [3]:
data.head()

,ACHV,FAM,PEER,SCHOOL
0,-0.43148,0.60814,0.03509,0.16607
1,0.79969,0.79369,0.47924,0.53356
2,-0.92467,-0.82630,-0.61951,-0.78635
3,-2.19081,-1.25310,-1.21675,-1.04076
4,-2.84818,0.17399,-0.18517,0.14229


In [4]:
model = smf.ols("ACHV~FAM+PEER+SCHOOL",data).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                   ACHV   R-squared:                       0.206
Model:                            OLS   Adj. R-squared:                  0.170
Method:                 Least Squares   F-statistic:                     5.717
Date:                Sun, 13 Sep 2020   Prob (F-statistic):            0.00153
Time:                        11:14:04   Log-Likelihood:                -148.20
No. Observations:                  70   AIC:                             304.4
Df Residuals:                      66   BIC:                             313.4
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.0700      0.251     -0.279      0.7

## 2.方差膨胀因子

In [5]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [23]:
vif1 = variance_inflation_factor(sm.add_constant(data[["FAM","PEER","SCHOOL"]].values),1)
vif2 = variance_inflation_factor(sm.add_constant(data[["FAM","PEER","SCHOOL"]].values),2)
vif3 = variance_inflation_factor(sm.add_constant(data[["FAM","PEER","SCHOOL"]].values),3)

In [24]:
vif1,vif2,vif3

(37.580639932204285, 30.21165657710285, 83.15543705009657)

In [25]:
test = smf.ols("FAM~PEER+SCHOOL",data).fit()
print(test.summary())

                            OLS Regression Results                            
Dep. Variable:                    FAM   R-squared:                       0.973
Model:                            OLS   Adj. R-squared:                  0.973
Method:                 Least Squares   F-statistic:                     1225.
Date:                Sun, 13 Sep 2020   Prob (F-statistic):           1.73e-53
Time:                        14:52:10   Log-Likelihood:                 22.514
No. Observations:                  70   AIC:                            -39.03
Df Residuals:                      67   BIC:                            -32.28
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0209      0.022      0.969      0.3

In [27]:
#vif1
1/(1-test.rsquared)

37.580639932204285

In [28]:
#删除变量以消除共线性是下策

## 3.条件指数

In [40]:
French_data = pd.read_csv("/Users/luofeifei/study/RegressionAnalysis/All_Data/P241.txt",sep="\t")
Fdata = French_data.loc[French_data["YEAR"]<=59,:]

In [42]:
cor = np.corrcoef(Fdata[["DOPROD","STOCK","CONSUM"]].values.T)

In [61]:
diag = np.linalg.eigvals(cor)
diag.sort()
diag = diag[::-1]
diag

array([1.99915493, 0.99815418, 0.00269089])

In [62]:
k2 = np.sqrt(diag[0]/diag[1])
k3 = np.sqrt(diag[0]/diag[2])
k2,k3

(1.4152214831664474, 27.25682703269725)

In [66]:
#特征向量
#["DOPROD","STOCK","CONSUM"]
#STOCK变量对应的特征值最小
np.linalg.eig(cor)

(array([1.99915493, 0.00269089, 0.99815418]),
 array([[ 0.70633041,  0.70698208,  0.03568867],
        [ 0.04350059,  0.00697079, -0.99902908],
        [ 0.70654444, -0.7071971 ,  0.02583046]]))